In [18]:
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from models.keras_ssd512 import ssd_512
from keras_loss_function.keras_ssd_loss import SSDLoss
from data_generator.object_detection_2d_data_generator import DataGenerator
from eval_utils.average_precision_evaluator_test import Evaluator
import numpy as np
import warnings
import os

ModuleNotFoundError: No module named 'cv2'

In [20]:
try:
    import h5py
except ImportError:
    warnings.warn("'h5py' module is missing. The fast HDF5 dataset option will be unavailable.")
try:
    import json
except ImportError:
    warnings.warn("'json' module is missing. The JSON-parser will be unavailable.")
try:
    from bs4 import BeautifulSoup
except ImportError:
    warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")
try:
    import pickle
except ImportError:
    warnings.warn(
        "'pickle' module is missing. You won't be able to save parsed file lists and annotations as pickled files.")

In [21]:
img_height = 512
img_width = 512
n_classes = 3
detection_mode = 'test'
model_mode = 'inference'
modelnames=['ssd512_2013_adam16_0.0001_time3_epoch-02_loss-10.9858_val_loss-10.1615.h5', 'ssd512_2013_adam16_0.0001_time3_epoch-01_loss-84.2419_val_loss-11.6939.h5']
modelindex=1
for modelname in modelnames:
    K.clear_session() # Clear previous models from memory.
    model = ssd_512(image_size=(img_height, img_width, 3),
                    n_classes=n_classes,
                    mode=model_mode,
                    l2_regularization=0.0005,
                    scales=[0.3, 0.15, 0.07, 0.04],
                    aspect_ratios_per_layer=[[1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                             [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                             [1.0, 2.0, 0.5, 3.0, 1.0/3.0]],
                    two_boxes_for_ar1=True,
                    steps=[16, 8, 4],
                    offsets=[0.5, 0.5, 0.5],
                    clip_boxes=False,
                    variances=[0.1, 0.1, 0.2, 0.2],
                    normalize_coords=True,
                    subtract_mean=[123, 117, 104],
                    swap_channels=[2, 1, 0],
                    confidence_thresh=0.01,
                    iou_threshold=0.45,
                    top_k=200,
                    nms_max_output_size=400)


NameError: name 'concatenate' is not defined

In [22]:
 weights_path = '/data/deeplearn/SWEIPENet/' + modelname
    model.load_weights(weights_path, by_name=True)
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
    model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

    dataset = DataGenerator()
    Pascal_VOC_dataset_images_dir = '/data/deeplearn/SWEIPENet/dataset/JPEGImages/'
    Pascal_VOC_dataset_annotations_dir = '/data/deeplearn/SWEIPENet/dataset/Annotations/'
    Pascal_VOC_dataset_image_set_filename = '/data/deeplearn/SWEIPENet/dataset/ImageSets/Main/test.txt'
    classes = ['background', 'seacucumber', 'seaurchin', 'scallop']
    dataset.parse_xml(images_dirs=[Pascal_VOC_dataset_images_dir],
                      image_set_filenames=[Pascal_VOC_dataset_image_set_filename],
                      annotations_dirs=[Pascal_VOC_dataset_annotations_dir],
                      classes=classes,
                      include_classes='all',
                      exclude_truncated=False,
                      exclude_difficult=False,
                      ret=False)

    evaluator = Evaluator(model=model,
                          modelindex=modelindex,
                          n_classes=n_classes,
                          data_generator=dataset,
                          model_mode=model_mode,
                          detection_mode=detection_mode)
    results = evaluator(img_height=img_height,
                        img_width=img_width,
                        batch_size=32,
                        data_generator_mode='resize',
                        round_confidences=False,
                        matching_iou_threshold=0.5,
                        border_pixels='include',
                        sorting_algorithm='quicksort',
                        average_precision_mode='sample',
                        num_recall_points=11,
                        ignore_neutral_boxes=True,
                        return_precisions=True,
                        return_recalls=True,
                        return_average_precisions=True,
                        verbose=True)
    modelindex=modelindex+1
print('Detection results of multiple models have been saved in /data/deeplearn/SWEIPENet/dataset/Detections/...')

IndentationError: unexpected indent (4068919234.py, line 2)

In [23]:
# Ensembel the results of multiple models
def intersection_area_(boxes1, boxes2, coords='corners', mode='outer_product', border_pixels='half'):
    '''
    The same as 'intersection_area()' but for internal use, i.e. without all the safety checks.
    '''
    m = boxes1.shape[0]  # The number of boxes in `boxes1`
    n = boxes2.shape[0]  # The number of boxes in `boxes2`
    # Set the correct coordinate indices for the respective formats.
    if coords == 'corners':
        xmin = 0
        ymin = 1
        xmax = 2
        ymax = 3
    elif coords == 'minmax':
        xmin = 0
        xmax = 1
        ymin = 2
        ymax = 3

    if border_pixels == 'half':
        d = 0
    elif border_pixels == 'include':
        d = 1  # If border pixels are supposed to belong to the bounding boxes, we have to add one pixel to any difference `xmax - xmin` or `ymax - ymin`.
    elif border_pixels == 'exclude':
        d = -1  # If border pixels are not supposed to belong to the bounding boxes, we have to subtract one pixel from any difference `xmax - xmin` or `ymax - ymin`.
